In [2]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI, MCSAPI

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("SWAN_API"), environment=os.getenv("SWAN_ENV"))

api_key = os.getenv("MCS_API")
mcs_api = MCSAPI(api_key)


In [3]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 73,
  'name': 'R1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 32 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '22.0',
  'status': 'available'}]

In [4]:
device = "G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

13
('G1ae.medium', 13, ['Quebec-CA', 'North Carolina-US'])


In [5]:
from swan.object.source_uri import LagrangeSpace

lag = LagrangeSpace('0x1898015fCde650CDB05a69FB9EC70C77e2145206', 'NSFW', os.getenv("WALLET"), obj)
 
lag.get_space_info()

# Upload Directory to MCS
response = lag.generate_source_uri('swan_test', 'mar8s1', 'nsfw.json', mcs_client = mcs_api)

In [6]:
print(lag.source_uri)

https://plutotest.acl.swanipfs.com/ipfs/Qmb39hJJ3mMmfDcVjt59m7SfC6ZimuQQsRHhfWctQaQAoz


In [6]:
import sys
sys.path.insert(0, '..')
 
import os
from dotenv import load_dotenv
load_dotenv()
 
from swan import SwanContract
 
pk = os.getenv('PK')
rpc = os.getenv('RPC')
 
c = SwanContract(pk, rpc)
 
# Test esimate lock revenue
r = c.estimate_payment(obj.id, 2)
print(r)

22000000000000000000


In [7]:
r = c._approve_swan_token(22000000000000000000)
print(r)
 
r = c.lock_revenue('1', obj.id, 1)
print(r)

0xfc27e23ae3d9785eeae90c0b9fa0a5910f1f32a631ec7799edf14d1841c790fd
0xa0ed91eb6fb8b1ab3b3ad112fd65ecf74aea5d6cbbddc70a3b059cb16af11c96


In [8]:
# Retrieve List of Hardwares

# for hardware in hardwares:
#     if hardware.name == 'C1ae.medium':
#         print(hardware.region, hardware.price)
        
# Deploy task
result = swan_api.deploy_task(
                              cfg_name=device, 
                              region='Quebec-CA', 
                              start_in=60*40, duration=3600*1, 
                              job_source_uri=lag.source_uri, 
                              paid=11,
                              tx_hash=r,
                              wallet_address=os.getenv('WALLET'),
                            )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

{'data': {'task': {'created_at': '1712084405', 'end_at': '1712090401', 'leading_job_id': None, 'status': 'created', 'task_detail_cid': 'https://plutotest.acl.swanipfs.com/ipfs/Qmdwj5D9skp52vW4mYEVp4yNHuJnx5cdF6byQwMk1pPy7c', 'updated_at': '1712084405', 'uuid': '72b34e63-2845-4dff-8913-47edce4d0d88'}}, 'message': 'Task_uuid created.', 'status': 'success'}
Task UUID: 72b34e63-2845-4dff-8913-47edce4d0d88


In [9]:
# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_uri = info['data']['jobs'][0]['job_result_uri']
        job_real_uri = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_uri)
        print("Job Real URL: ", job_real_uri)
        
        # break
        if status == 'deployToK8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(30)

KeyboardInterrupt: 

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

In [3]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}
json_data = {
    'link': [
        ##### NSFW
        'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s',
        'https://cdni.pornpics.de/460/7/622/16870332/16870332_155_7baa.jpg',
        'https://cdn77-pic.xvideos-cdn.com/videos/thumbs169poster/28/56/d5/2856d555486fba22716a930e8a928c7f-2/2856d555486fba22716a930e8a928c7f.30.jpg',
        'https://cdn.pornapi.online/content/2023-06-15/eGOkYIoH.jpg',
        'https://porn62.com/wp-content/uploads/2020/10/xxx-porn.jpg',
        'https://static-ca-cdn.eporner.com/gallery/58/U0/pvyTrBuU058/11679702-ai-porn-bbw-image-5.jpg',
        ##### Safe
        'https://plutotest.acl.swanipfs.com/ipfs/QmbxYuLWdyQQpdK9wKqK4A8TiFtMf885d6mHrfvaf6CZyX?filename=istockphoto-157030584-612x612.jpg',
        'https://plutotest.acl.swanipfs.com/ipfs/QmRACojSdFuqnyyfQZ9Zgiz6zrVCUX1JRkYZyvRGu1MCzG?filename=Ipfs-logo-1024-ice-text.png',
        'https://plutotest.acl.swanipfs.com/ipfs/QmSeUJZYyC2UVJK2oH7HzVvd3XgEFPtkoiZWSQ6YzSKP36?filename=Happy-Guy.jpg',
        'https://plutotest.acl.swanipfs.com/ipfs/QmUhnnxBNP2tmYAFaBKYqem5weS9H4jXQcjwT7kvSWgXYV?filename=nm-how-happiness-affects-health-tnail.jpg',
    ],
}
# https://1j2muw1lae.dev2.crosschain.computer
# https://u3xc9xrzmv.dev2.crosschain.computer/
response = requests.post(f"https://lrltc5cerm.cp.filezoo.com.cn/link", headers=headers, json=json_data)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


Expecting value: line 1 column 1 (char 0)
<Response [404]>


In [49]:
log_path = '../nsfw_imgs.csv'

if not os.path.isfile(log_path):
    with open(log_path, 'w') as f:
        f.write('cid,link,IS NSFW IMAGE,Probability\n')

with open(log_path, 'a') as f:
    for result in response.json()['result']:
        if result['is_nsfw_image']:
            f.write(f"{result['cid']},{result['link']},{result['is_nsfw_image']},{result['probability']}\n")
